# Data Cleaning and Preparation

In [38]:
import numpy as np
import pandas as pd


## Data Transformation (continue)
So far in this lesson we’ve been concerned with rearranging data. Filtering, cleaning,
and other transformations are another class of important operations.

### Detecting and Filtering Outliers
Filtering or transforming outliers is largely a matter of applying array operations.
Consider a DataFrame with some normally distributed data

In [39]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.006338,-0.003542,0.015041,0.020466
std,0.986199,1.008984,0.980168,0.929983
min,-3.342739,-2.816576,-3.571263,-3.329085
25%,-0.706116,-0.697805,-0.644623,-0.593368
50%,-0.036128,0.003621,0.033260,0.059341
75%,0.697866,0.638397,0.649911,0.619700
max,3.612420,3.168776,3.471025,2.963320


In [40]:
# find values in one of the columns exceeding 3 in absolute value
data[data[0].abs() > 3]

,0,1,2,3
425,-3.342739,1.793226,-1.544512,0.381850
638,3.612420,1.502677,0.501882,-1.037982


In [41]:
# select all rows having a value exceeding 3 or –3
data[(data.abs() > 3).any(axis=1)]

,0,1,2,3
71,-0.336141,3.127005,1.044151,0.059658
180,0.496789,0.319366,-3.571263,1.741858
325,-0.655961,-0.226270,-3.269214,-0.371205
425,-3.342739,1.793226,-1.544512,0.381850
638,3.612420,1.502677,0.501882,-1.037982
693,1.262268,-0.136784,0.332245,-3.329085
764,0.899012,0.410353,3.304725,-0.766152
884,0.305083,0.946548,3.057462,0.811992
887,1.114411,0.077214,3.471025,-0.715580
961,0.635029,3.168776,0.654580,1.351346


In [42]:
# set outliers to 3 or -3 depending on its sign
# data[(data.abs() > 3).any(axis=1)] = np.sign(data) * 3

In [43]:
# data[(data.abs() > 3).any(axis=1)]
data.iloc[71, :]

0   -3.0
1    3.0
2    3.0
3    3.0
Name: 71, dtype: float64

In [32]:

def normalize_values(row):
    for i, value in enumerate(row):
        if value > 3: 
            row[i] = 3
        elif value < -3:
            row[i] = -3
    
    return row

temp = data.apply(normalize_values)
temp.iloc[64, :]

0    3.000000
1   -1.117346
2   -1.766104
3    0.108443
Name: 64, dtype: float64

### Computing Indicator/Dummy Variables
Another type of transformation for statistical modeling or machine learning applica‐
tions is converting a categorical variable into a “dummy” or “indicator” matrix.

column in a DataFrame has k distinct values, you would derive a matrix or Data‐
Frame with k columns containing all 1s and 0s.

pandas has a `get_dummies` function for doing this

In [33]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [34]:
# create dummy variables for column 'key'
pd.get_dummies(df["key"])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [35]:
# create dummy variables for column 'key'  then add the columns to a variable
temp = pd.get_dummies(df["key"], prefix="grade")
temp

,grade_a,grade_b,grade_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [37]:
# join the dummies to the Data-Frame
df.join(temp).drop(columns=["key"])

,data1,grade_a,grade_b,grade_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [44]:
# read the dataset 'movies.dat' and display the first 10 rows of it

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('movies.dat', sep='::', engine="python",
                       header=None, names=mnames)
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Adding dummy variables for each genre requires a little bit of wrangling.

In [45]:
# make a list 'genres' contains all distict genres
genres = []
for genre in movies["genres"]:
    genres.extend(genre.split("|"))

genres = pd.unique(genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [47]:
# create a DataFrame 'dummies' of size (#movies, #genres) and fill it with zeros
matrix = np.zeros((len(movies), len(genres)), dtype=int)
dummies = pd.DataFrame(matrix, columns=genres)
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3880,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3881,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [54]:
# get the genres from the first row of 'movies' DataFrame, then get their indices in the 'dummies' DataFrame
# hint: use 'get_indexer' method
temp = movies.loc[0, "genres"]
dummies.columns.get_indexer(temp.split("|"))

array([0, 1, 2], dtype=int64)

In [55]:
# for each value of 'genre' column in 'movies' DataFrame, set the corresponding columns in dummies to 1 
for i, genre in enumerate(movies["genres"]):
    col_idx = dummies.columns.get_indexer(genre.split("|"))
    dummies.iloc[i, col_idx] = 1

dummies


,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3880,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3881,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [57]:
# join the DataFrames 'movies' and 'dummies', then display the first row 
movies.join(dummies).drop(columns="genres")


,movie_id,title,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1,Toy Story (1995),1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,3949,Requiem for a Dream (2000),0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3880,3950,Tigerland (2000),0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3881,3951,Two Family House (2000),0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


## String Manipulation
Python has long been a popular raw data manipulation language in part due to its
ease of use for string and text processing. Most text operations are made simple with
the string object’s **built-in methods**. 

For more complex pattern matching and text manipulations, **regular expressions** may be needed. 

**pandas** adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data,
additionally handling the annoyance of missing data.

### String Object Methods

In [59]:
# convert the string to a list using the ',' as separator
val = 'a,b,  guido'
val.split(",")

['a', 'b', '  guido']

In [60]:
# convert the string to a list using the ',' as separator and remove the extra spaces
[v.strip() for v in val.split(",")]

['a', 'b', 'guido']

In [62]:
pieces = ["Omar", "Karem", "Mahmoud"]

In [63]:
# concatenate the parts back with separator '::'
first, second, third = pieces
first + '::' + second + '::' + third

'Omar::Karem::Mahmoud'

In [64]:
# concatenate the parts back with separator '::'
"::".join(pieces)

'Omar::Karem::Mahmoud'

**check**: try the `in` operator and str methods `index`, `find`, `count` and `replace`

![](assets/built-in-str-methods.png)

### Regular Expressions
Regular expressions provide a flexible way to search or match (often more complex)
string patterns in text. A single expression, commonly called a regex, is a string
formed according to the regular expression language.

The `re` module functions fall into three categories: **pattern matching**, **substitution**,
and **splitting**.

In [65]:
import re
text = "foo    bar\t baz  \tqux"


suppose we wanted to split a string with a variable number of whitespace characters
(tabs, spaces, and newlines). The regex describing one or more whitespace characters
is \s+:

In [67]:
# split the string depending on the whitespaces
# text.split(" ")
re.split("\s+", text)

['foo', 'bar', 'baz', 'qux']

In [68]:
# find all the string depending on the whitespaces
re.findall("\s+", text)


['    ', '\t ', '  \t']

**Note:** Creating a regex object with `re.compile` is highly recommended if you intend to
apply the same expression to many strings; doing so will save CPU cycles

In [69]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [70]:
# get a list of all emails in the text
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

Relatedly, `sub` will return a new string with occurrences of the pattern replaced by the
a new string:


In [71]:
regex.sub("[email]", text)

'Dave [email]\nSteve [email]\nRob [email]\nRyan [email]\n'

Suppose you wanted to find email addresses and simultaneously segment each
address into its three components: *username*, *domain name*, and *domain suffix*.

In [82]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [73]:
# find all email in the text
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [78]:
# prefix each segment of the email with a suitable label
print(regex.sub("User: \\1, Domain: \\2, Ext: \\3", text))

Dave User: dave, Domain: google, Ext: com
Steve User: steve, Domain: gmail, Ext: com
Rob User: rob, Domain: gmail, Ext: com
Ryan User: ryan, Domain: yahoo, Ext: com



![](assets/re-methods.png)

### Vectorized String Functions in pandas

In [79]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data


Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

Series has array-oriented methods for string operations that skip NA values. These are accessed through Series’s **str attribute**

In [80]:
# check if the email is gmail
data.str.match("\w+@gmail.com")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Regular expressions can be used, too, along with any re options like IGNORECASE

In [84]:
# using the pattern declared earlier, find all parts of each email
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [85]:
# use the match method to check if the field matches an email or not
data.str.match(pattern, flags=re.IGNORECASE)

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [86]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


![](assets/series-str-methods.png)

# Data Aggregation and Group Operations

## GroupBy Mechanics

the term split-apply-combine is used for describing group operations. 

- In the first stage of the process, data contained in a pandas object, whether a Series, Data‐Frame, or otherwise, is **split** into groups based on one or more keys that you provide.
- Once this is done, a function is **applied** to each group, producing a new value. 
- Finally, the results of all those function applications are **combined** into a result object. 

The form of the resulting object will usually depend on what’s being done to the data.

![](assets/group-aggregation.png)

In [87]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randint(0, 10, 5),
                   'data2' : np.random.randint(0, 10, 5)})
df

,key1,key2,data1,data2
0,a,one,6,1
1,a,two,3,2
2,b,one,1,4
3,b,two,8,9
4,a,one,6,7


In [89]:
# group data of column 'data1' by 'key1' then print the groups
groups = df["data1"].groupby(df["key1"])
groups.groups

{'a': [0, 1, 4], 'b': [2, 3]}

In [90]:
# calculate the mean in each group
groups.mean()

key1
a    5.0
b    4.5
Name: data1, dtype: float64

In [91]:
# group data of column 'data1' by 'key1' and 'key2' then print the groups
groups = df["data1"].groupby([df["key1"], df["key2"]])
groups.groups

{('a', 'one'): [0, 4], ('a', 'two'): [1], ('b', 'one'): [2], ('b', 'two'): [3]}

In [95]:
# calculate the mean in each group
means = groups.mean()
means

key1  key2
a     one     6
      two     3
b     one     1
      two     8
Name: data1, dtype: int32

In [96]:
means.loc[("a", "one")]

6

In [98]:
# unstack the result Series
means = means.unstack()
means

key2,one,two
key1,,
a,6,3
b,1,8


In [99]:
# try some selection on the result DataFrame
means.loc["a", "one"]

6

### Iterating Over Groups
The GroupBy object supports iteration, generating a sequence of **2-tuples** containing
the **group name** along with the **chunk of data**.

In [100]:
# group data of DataFrame 'data' by 'key1' then print each group name and data
groups = df["data1"].groupby(df["key1"])
for name, group in groups:
    print(name)
    print(group)
    print("-" * 20)


a
0    6
1    3
4    6
Name: data1, dtype: int32
--------------------
b
2    1
3    8
Name: data1, dtype: int32
--------------------


In [101]:
# group data of DataFrame 'data' by 'key1' and 'key2' then print each group name and data
groups = df["data1"].groupby([df["key1"], df["key2"]])
for name, group in groups:
    print(name)
    print(group)
    print("-" * 20)

('a', 'one')
0    6
4    6
Name: data1, dtype: int32
--------------------
('a', 'two')
1    3
Name: data1, dtype: int32
--------------------
('b', 'one')
2    1
Name: data1, dtype: int32
--------------------
('b', 'two')
3    8
Name: data1, dtype: int32
--------------------


In [104]:
# group data of DataFrame 'data' by 'key1' then convert it to a dictionary of DataFrames
groups = df["data1"].groupby(df["key1"])
groups = dict(list(groups))
# groups["a"]
groups

{'a': 0    6
 1    3
 4    6
 Name: data1, dtype: int32,
 'b': 2    1
 3    8
 Name: data1, dtype: int32}

In [106]:
# group the columns by its datatypes, then print the groups
groups = df.groupby(df.dtypes, axis=1)
groups.groups

{int32: ['data1', 'data2'], object: ['key1', 'key2']}

In [107]:
# loop through the groups and print it
for name, group in groups:
    print(name)
    print(group)
    print("-" * 20)

int32
   data1  data2
0      6      1
1      3      2
2      1      4
3      8      9
4      6      7
--------------------
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one
--------------------


### Selecting a Column or Subset of Columns
Indexing a GroupBy object created from a DataFrame with a column name or array
of column names has the effect of column subsetting for aggregation. This means
that:
```python
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
```
are syntactic sugar for:
```python
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])
```


In [114]:
# group data of DataFrame 'df' by 'key1' then calculate the mean of column 'data2'
df.groupby("key1")["data2"].mean()
# df["data2"].groupby(df["key1"]).mean()

key1
a    3.333333
b    6.500000
Name: data2, dtype: float64

In [116]:
# How to get result as DataFrame GroupBy or Series GroupBy?
df.groupby("key1")[["data2"]].mean()


,data2
key1,
a,3.333333
b,6.500000


### Grouping with Dicts and Series

In [117]:
people = pd.DataFrame(np.random.randint(0, 10, (5,5)),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,9,5.0,7.0,9,1
Steve,6,8.0,4.0,9,3
Wes,5,NaN,NaN,0,7
Jim,6,9.0,6.0,1,3
Travis,1,7.0,7.0,6,1


In [118]:
mapping = {'Joe': 'red', 'Steve': 'red', 'Wes': 'blue',
           'Jim': 'blue', 'Travis': 'red', 'Elon' : 'orange'}

In [119]:
# group and sum the scores of teams red and blue
people.groupby(mapping).sum()


,a,b,c,d,e
blue,11,9.0,6.0,1,10
red,16,20.0,18.0,24,5


In [121]:
# convert the dict to a Series, then group and count the scores of teams red and blue
mapping = pd.Series(mapping)
people.groupby(mapping).sum()


,a,b,c,d,e
blue,11,9.0,6.0,1,10
red,16,20.0,18.0,24,5


### Grouping with Functions

In [122]:
people.groupby(len).sum()


,a,b,c,d,e
3,20,14.0,13.0,10,11
5,6,8.0,4.0,9,3
6,1,7.0,7.0,6,1


## independent Practice:
- read the dataset `tips.csv`
- create a new column 'tip_pct', which is tip / total_bill
- replace the short-day name with the full-day name and convert it to upper-case
- calculate the average tip percent for smokers and non-smokers
- calculate the max and average tip percent for each time
- calculate the average tip percent for each day and time
- create dummy variables for the day and time columns


